In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

In [3]:
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced

# Read the CSV and Perform Basic Data Cleaning

In [4]:
# https://help.lendingclub.com/hc/en-us/articles/215488038-What-do-the-different-Note-statuses-mean-

columns = [
    "loan_amnt", "int_rate", "installment", "home_ownership",
    "annual_inc", "verification_status", "issue_d", "loan_status",
    "pymnt_plan", "dti", "delinq_2yrs", "inq_last_6mths",
    "open_acc", "pub_rec", "revol_bal", "total_acc",
    "initial_list_status", "out_prncp", "out_prncp_inv", "total_pymnt",
    "total_pymnt_inv", "total_rec_prncp", "total_rec_int", "total_rec_late_fee",
    "recoveries", "collection_recovery_fee", "last_pymnt_amnt", "next_pymnt_d",
    "collections_12_mths_ex_med", "policy_code", "application_type", "acc_now_delinq",
    "tot_coll_amt", "tot_cur_bal", "open_acc_6m", "open_act_il",
    "open_il_12m", "open_il_24m", "mths_since_rcnt_il", "total_bal_il",
    "il_util", "open_rv_12m", "open_rv_24m", "max_bal_bc",
    "all_util", "total_rev_hi_lim", "inq_fi", "total_cu_tl",
    "inq_last_12m", "acc_open_past_24mths", "avg_cur_bal", "bc_open_to_buy",
    "bc_util", "chargeoff_within_12_mths", "delinq_amnt", "mo_sin_old_il_acct",
    "mo_sin_old_rev_tl_op", "mo_sin_rcnt_rev_tl_op", "mo_sin_rcnt_tl", "mort_acc",
    "mths_since_recent_bc", "mths_since_recent_inq", "num_accts_ever_120_pd", "num_actv_bc_tl",
    "num_actv_rev_tl", "num_bc_sats", "num_bc_tl", "num_il_tl",
    "num_op_rev_tl", "num_rev_accts", "num_rev_tl_bal_gt_0",
    "num_sats", "num_tl_120dpd_2m", "num_tl_30dpd", "num_tl_90g_dpd_24m",
    "num_tl_op_past_12m", "pct_tl_nvr_dlq", "percent_bc_gt_75", "pub_rec_bankruptcies",
    "tax_liens", "tot_hi_cred_lim", "total_bal_ex_mort", "total_bc_limit",
    "total_il_high_credit_limit", "hardship_flag", "debt_settlement_flag"
]

target = ["loan_status"]

In [5]:
# Load the data
file_path = Path('LoanStats_2019Q1.csv')
df = pd.read_csv(file_path, skiprows=1)[:-2]
df = df.loc[:, columns].copy()

# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')

# Drop the null rows
df = df.dropna()

# Remove the `Issued` loan status
issued_mask = df['loan_status'] != 'Issued'
df = df.loc[issued_mask]

# convert interest rate to numerical
df['int_rate'] = df['int_rate'].str.replace('%', '')
df['int_rate'] = df['int_rate'].astype('float') / 100


# Convert the target column values to low_risk and high_risk based on their values
x = {'Current': 'low_risk'}   
df = df.replace(x)

x = dict.fromkeys(['Late (31-120 days)', 'Late (16-30 days)', 'Default', 'In Grace Period'], 'high_risk')    
df = df.replace(x)

df.reset_index(inplace=True, drop=True)

df.head()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,dti,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
0,10500.0,0.1719,375.35,RENT,66000.0,Source Verified,Mar-2019,low_risk,n,27.24,...,85.7,100.0,0.0,0.0,65687.0,38199.0,2000.0,61987.0,N,N
1,25000.0,0.2000,929.09,MORTGAGE,105000.0,Verified,Mar-2019,low_risk,n,20.23,...,91.2,50.0,1.0,0.0,271427.0,60641.0,41200.0,49197.0,N,N
2,20000.0,0.2000,529.88,MORTGAGE,56000.0,Verified,Mar-2019,low_risk,n,24.26,...,66.7,50.0,0.0,0.0,60644.0,45684.0,7500.0,43144.0,N,N
3,10000.0,0.1640,353.55,RENT,92000.0,Verified,Mar-2019,low_risk,n,31.44,...,100.0,50.0,1.0,0.0,99506.0,68784.0,19700.0,76506.0,N,N
4,22000.0,0.1474,520.39,MORTGAGE,52000.0,Not Verified,Mar-2019,low_risk,n,18.76,...,100.0,0.0,0.0,0.0,219750.0,25919.0,27600.0,20000.0,N,N


In [6]:
# Make our target variable numeric by using labelEncoder
from sklearn.preprocessing import LabelEncoder

In [7]:
le = LabelEncoder()

In [8]:
# apply this encoder to the loan_status column
df['loan_status'] = le.fit_transform(df['loan_status'])

In [9]:
df.head()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,dti,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
0,10500.0,0.1719,375.35,RENT,66000.0,Source Verified,Mar-2019,1,n,27.24,...,85.7,100.0,0.0,0.0,65687.0,38199.0,2000.0,61987.0,N,N
1,25000.0,0.2000,929.09,MORTGAGE,105000.0,Verified,Mar-2019,1,n,20.23,...,91.2,50.0,1.0,0.0,271427.0,60641.0,41200.0,49197.0,N,N
2,20000.0,0.2000,529.88,MORTGAGE,56000.0,Verified,Mar-2019,1,n,24.26,...,66.7,50.0,0.0,0.0,60644.0,45684.0,7500.0,43144.0,N,N
3,10000.0,0.1640,353.55,RENT,92000.0,Verified,Mar-2019,1,n,31.44,...,100.0,50.0,1.0,0.0,99506.0,68784.0,19700.0,76506.0,N,N
4,22000.0,0.1474,520.39,MORTGAGE,52000.0,Not Verified,Mar-2019,1,n,18.76,...,100.0,0.0,0.0,0.0,219750.0,25919.0,27600.0,20000.0,N,N


In [10]:
df['loan_status'].value_counts()

1    68470
0      347
Name: loan_status, dtype: int64

In [11]:
df.shape

(68817, 86)

In [12]:
# check datatypes of all columns and values both numeric not text or object
df.dtypes

loan_amnt                     float64
int_rate                      float64
installment                   float64
home_ownership                 object
annual_inc                    float64
                               ...   
total_bal_ex_mort             float64
total_bc_limit                float64
total_il_high_credit_limit    float64
hardship_flag                  object
debt_settlement_flag           object
Length: 86, dtype: object

In [13]:
# some columns data type is object need to change to number 
columns_to_be_fixed = []

for d in df.columns:
#     print(d, df[d].dtype)
    if df[d].dtype == 'object':
        print(d)
        columns_to_be_fixed.append(d)

home_ownership
verification_status
issue_d
pymnt_plan
initial_list_status
next_pymnt_d
application_type
hardship_flag
debt_settlement_flag


In [14]:
columns_to_be_fixed

['home_ownership',
 'verification_status',
 'issue_d',
 'pymnt_plan',
 'initial_list_status',
 'next_pymnt_d',
 'application_type',
 'hardship_flag',
 'debt_settlement_flag']

In [15]:
# get_dummies this  will help all columns supplied in column parameter to change as number 
df = pd.get_dummies(df, columns=columns_to_be_fixed)

In [17]:
df.dtypes

loan_amnt                      float64
int_rate                       float64
installment                    float64
annual_inc                     float64
loan_status                      int32
                                ...   
next_pymnt_d_May-2019            uint8
application_type_Individual      uint8
application_type_Joint App       uint8
hardship_flag_N                  uint8
debt_settlement_flag_N           uint8
Length: 96, dtype: object

In [18]:
df.head()

,loan_amnt,int_rate,installment,annual_inc,loan_status,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,...,issue_d_Mar-2019,pymnt_plan_n,initial_list_status_f,initial_list_status_w,next_pymnt_d_Apr-2019,next_pymnt_d_May-2019,application_type_Individual,application_type_Joint App,hardship_flag_N,debt_settlement_flag_N
0,10500.0,0.1719,375.35,66000.0,1,27.24,0.0,0.0,8.0,0.0,...,1,1,0,1,0,1,1,0,1,1
1,25000.0,0.2000,929.09,105000.0,1,20.23,0.0,0.0,17.0,1.0,...,1,1,0,1,0,1,1,0,1,1
2,20000.0,0.2000,529.88,56000.0,1,24.26,0.0,0.0,8.0,0.0,...,1,1,0,1,0,1,1,0,1,1
3,10000.0,0.1640,353.55,92000.0,1,31.44,0.0,1.0,10.0,1.0,...,1,1,0,1,0,1,1,0,1,1
4,22000.0,0.1474,520.39,52000.0,1,18.76,0.0,1.0,14.0,0.0,...,1,1,0,1,0,1,1,0,1,1



# Split the Data into Training and Testing

In [19]:
# Create our features
# Create our target
y = df["loan_status"]
X = df.drop(columns ="loan_status")

In [20]:
# Check the balance of our target values
y.value_counts()

1    68470
0      347
Name: loan_status, dtype: int64

In [21]:
# Split our data for 
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y, random_state=1)

In [24]:
#scale the data, especially when comparing the performances of different models
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_scaler = scaler.fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

# Ensemble Learners

In this section, you will compare two ensemble algorithms to determine which algorithm results in the best performance. You will train a Balanced Random Forest Classifier and an Easy Ensemble AdaBoost classifier . For each algorithm, be sure to complete the folliowing steps:

1. Train the model using the training data. 
2. Calculate the balanced accuracy score from sklearn.metrics.
3. Print the confusion matrix from sklearn.metrics.
4. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.
5. For the Balanced Random Forest Classifier onely, print the feature importance sorted in descending order (most important feature to least important) along with the feature score

Note: Use a random state of 1 for each algorithm to ensure consistency between tests

### Balanced Random Forest Classifier

In [26]:
# Resample the training data with the BalancedRandomForestClassifier
from imblearn.ensemble import BalancedRandomForestClassifier
brfc = BalancedRandomForestClassifier(max_depth=2, random_state=1)
brfc.fit(X_train_scaled, y_train)  

BalancedRandomForestClassifier(max_depth=2, random_state=1)

In [27]:
y_pred = brfc.predict(X_test_scaled)

In [28]:
from sklearn.metrics import accuracy_score

In [29]:
accuracy_score(y_test, y_pred)

0.9493751816332462

In [30]:
cm = confusion_matrix(y_test, y_pred)
print(cm)

[[   51    50]
 [  821 16283]]


In [52]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

          0       0.06      0.50      0.95      0.10      0.69      0.46       101
          1       1.00      0.95      0.50      0.97      0.69      0.50     17104

avg / total       0.99      0.95      0.51      0.97      0.69      0.50     17205



In [31]:
print(brfc.feature_importances_)

[0.00492267 0.04945603 0.00300798 0.01013413 0.00735303 0.00484624
 0.0067339  0.0018179  0.         0.01674953 0.00028072 0.00874846
 0.02721177 0.10961587 0.12236813 0.14224597 0.07934436 0.01532335
 0.         0.         0.08742552 0.00328188 0.         0.
 0.         0.00922192 0.0186362  0.         0.00327952 0.00114491
 0.02001547 0.01364321 0.00432944 0.         0.00741705 0.00444314
 0.00542397 0.0201461  0.00857964 0.00196693 0.00676507 0.
 0.00336576 0.00091797 0.00474055 0.         0.         0.01676907
 0.00496773 0.         0.00259687 0.00242227 0.         0.01475889
 0.         0.         0.01431712 0.         0.00359704 0.0019507
 0.00442437 0.00685533 0.00450767 0.00288121 0.         0.
 0.         0.00928101 0.00330419 0.         0.         0.
 0.01007484 0.00798131 0.01862806 0.00335947 0.         0.
 0.00637863 0.         0.         0.         0.         0.00567703
 0.01294289 0.00452162 0.         0.         0.00289836 0.
 0.         0.         0.         0.        

In [32]:
X_test.columns

Index(['loan_amnt', 'int_rate', 'installment', 'annual_inc', 'dti',
       'delinq_2yrs', 'inq_last_6mths', 'open_acc', 'pub_rec', 'revol_bal',
       'total_acc', 'out_prncp', 'out_prncp_inv', 'total_pymnt',
       'total_pymnt_inv', 'total_rec_prncp', 'total_rec_int',
       'total_rec_late_fee', 'recoveries', 'collection_recovery_fee',
       'last_pymnt_amnt', 'collections_12_mths_ex_med', 'policy_code',
       'acc_now_delinq', 'tot_coll_amt', 'tot_cur_bal', 'open_acc_6m',
       'open_act_il', 'open_il_12m', 'open_il_24m', 'mths_since_rcnt_il',
       'total_bal_il', 'il_util', 'open_rv_12m', 'open_rv_24m', 'max_bal_bc',
       'all_util', 'total_rev_hi_lim', 'inq_fi', 'total_cu_tl', 'inq_last_12m',
       'acc_open_past_24mths', 'avg_cur_bal', 'bc_open_to_buy', 'bc_util',
       'chargeoff_within_12_mths', 'delinq_amnt', 'mo_sin_old_il_acct',
       'mo_sin_old_rev_tl_op', 'mo_sin_rcnt_rev_tl_op', 'mo_sin_rcnt_tl',
       'mort_acc', 'mths_since_recent_bc', 'mths_since_recent_in

In [33]:
sorted_results = sorted(brfc.feature_importances_, reverse=True)

In [34]:
for f in range(len(X_test.columns)):
    print(X_test.columns[f], sorted_results[f])

loan_amnt 0.14224596528606007
int_rate 0.12236813467280289
installment 0.1096158743815982
annual_inc 0.08742551915990909
dti 0.07934436277729512
delinq_2yrs 0.0494560302632254
inq_last_6mths 0.027211767847635987
open_acc 0.02014610229028629
pub_rec 0.020015471741778465
revol_bal 0.018636203018802013
total_acc 0.018628061366125218
out_prncp 0.016769065742773107
out_prncp_inv 0.01674952609030327
total_pymnt 0.01532334819200845
total_pymnt_inv 0.014758893818032614
total_rec_prncp 0.014317116687049061
total_rec_int 0.013643212235623823
total_rec_late_fee 0.012942893450228723
recoveries 0.010134128083062976
collection_recovery_fee 0.010074844098932816
last_pymnt_amnt 0.009281008130177196
collections_12_mths_ex_med 0.00922192276823999
policy_code 0.008748461728351116
acc_now_delinq 0.008579635169533775
tot_coll_amt 0.007981313718519135
tot_cur_bal 0.0074170525115990595
open_acc_6m 0.007353032552256988
open_act_il 0.006855328287224694
open_il_12m 0.0067650692923950815
open_il_24m 0.0067339031

### Easy Ensemble AdaBoost Classifier

In [42]:
# Train the  EasyEnsembleClassifier AdaBooster 

from sklearn.ensemble import AdaBoostClassifier

enadboo = AdaBoostClassifier(n_estimators=100,random_state=1)

In [43]:
enadboo.fit(X_train_scaled, y_train)

AdaBoostClassifier(n_estimators=100, random_state=1)

In [44]:
y_predi = enadboo.predict(X_test_scaled)

In [49]:
# Calculated the balanced accuracy score
y_predi = enadboo.predict(X_test_scaled) 
acc_score = accuracy_score(y_test, y_predi)
print(f"Accuracy Score : {acc_score}")

Accuracy Score : 0.9960476605637896


In [50]:
# Display the confusion matrix
cm = confusion_matrix(y_test, y_predi)
cm

array([[   38,    63],
       [    5, 17099]], dtype=int64)

In [51]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, y_predi))

                   pre       rec       spe        f1       geo       iba       sup

          0       0.88      0.38      1.00      0.53      0.61      0.35       101
          1       1.00      1.00      0.38      1.00      0.61      0.40     17104

avg / total       1.00      1.00      0.38      1.00      0.61      0.40     17205

